In [1]:
from __future__ import division
from __future__ import print_function
import pandas as pd
from sklearn.model_selection import train_test_split
from pyod.models.combination import aom, moa, average, maximization
from pyod.models.iforest import IForest
from pyod.models.ecod import ECOD
from sklearn.model_selection import train_test_split
from pyod.utils.data import generate_data
import numpy as np
from pyod.utils.data import evaluate_print
from pyod.utils.example import visualize
from pyod.version import *

In [24]:
df=pd.read_csv(r'train.csv',index_col='PassengerId')



In [25]:
model=ECOD()

In [26]:
df=df[['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked','Survived']]

In [27]:
for colname in df.select_dtypes("object"):
    df[colname], _ = df[colname].factorize()

In [28]:
df['Age']=df['Age'].fillna(df['Age'].mean())

In [29]:
df

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Survived
PassengerId,,,,,,,,
1,3,0,22.000000,1,0,7.2500,0,0
2,1,1,38.000000,1,0,71.2833,1,1
3,3,1,26.000000,0,0,7.9250,0,1
4,1,1,35.000000,1,0,53.1000,0,1
5,3,0,35.000000,0,0,8.0500,0,0
...,...,...,...,...,...,...,...,...
887,2,0,27.000000,0,0,13.0000,0,0
888,1,1,19.000000,0,0,30.0000,0,1
889,3,1,29.699118,1,2,23.4500,0,0


In [30]:

df['S_ECOD']=model.fit_predict(df)


c:\Python310\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function fit_predict is deprecated
  warnings.warn(msg, category=FutureWarning)


In [31]:
df

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Survived,S_ECOD
PassengerId,,,,,,,,,
1,3,0,22.000000,1,0,7.2500,0,0,0
2,1,1,38.000000,1,0,71.2833,1,1,0
3,3,1,26.000000,0,0,7.9250,0,1,0
4,1,1,35.000000,1,0,53.1000,0,1,0
5,3,0,35.000000,0,0,8.0500,0,0,0
...,...,...,...,...,...,...,...,...,...
887,2,0,27.000000,0,0,13.0000,0,0,0
888,1,1,19.000000,0,0,30.0000,0,1,0
889,3,1,29.699118,1,2,23.4500,0,0,0


In [32]:
from pyod.models.iforest import IsolationForest

iforest = IsolationForest(verbose=1).fit(df)
df['S_IForest']=iforest.predict(df)
df

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Survived,S_ECOD,S_IForest
PassengerId,,,,,,,,,,
1,3,0,22.000000,1,0,7.2500,0,0,0,1
2,1,1,38.000000,1,0,71.2833,1,1,0,1
3,3,1,26.000000,0,0,7.9250,0,1,0,1
4,1,1,35.000000,1,0,53.1000,0,1,0,1
5,3,0,35.000000,0,0,8.0500,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...
887,2,0,27.000000,0,0,13.0000,0,0,0,1
888,1,1,19.000000,0,0,30.0000,0,1,0,1
889,3,1,29.699118,1,2,23.4500,0,0,0,-1
